# Concevoir une application au service de la santé publique

En nous basant sur les informations nutritionnelles des aliements, nous allons concevoire une applications qui nous permettras de segmenter les aliments. Les étapes de construction de notre application sont:
- Sélection de variables pertinentes à notre étude
- Analyse univariée des variables pertinentes
- Traitement des valeurs manquantes et outliers
- Etude de la corrélation entre nos variables
- Standardisation des variables d'intérêt
- Appliquer l'analyse en composante principale pour la réduction de dimention afin de visualiser les données
- Application de l'algorithme du K-mean pour la segmentation.


Dans cette Notebook, nous procéderons à la sélection des variables pertinentes à notre étude et à la gestion des observations doublons.

In [1]:
import numpy as np
import pandas as pd

## I. Chargement de la base des données

In [3]:
data = pd.read_csv("D:/IML/project2/DATA/OpenFoodFact.csv", sep="\t")

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,24,25,26,28,44,49,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
data.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000000003327986,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1574175736,2019-11-19T15:02:16Z,1574175737,2019-11-19T15:02:17Z,Filetes de pollo empanado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000100,http://world-en.openfoodfacts.org/product/0000...,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,NaN,100g,...,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00000000001111111111,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1560020173,2019-06-08T18:56:13Z,1560020173,2019-06-08T18:56:13Z,Sfiudwx,NaN,dgesc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
data.shape

(1386297, 181)

Notre base de données originale comporte 1.386.297 observations et 181 variables.

## II. Suppression des doublons dans notre base de données

Nous allons vérifier si notre base de données ne comporte pas des doublons afin de les supprimer. Les doublons alourdissent notre base de données et n'apportent pas une nouvelle information à notre modèle.

In [131]:
data = data[~data.duplicated()]

Après suppression de doublons, nous passons de 1.386.297 à 1.386.245 aliments.

## III. Sélection de variables pertinentes à notre étude

Comme notre application se base uniquement sur les informations nutritionnelles des aliments pour la segmentation alors nous ne concidèrerons que les 111 variables qui y sont associées.

In [132]:
VariblesPertinentes = data.columns.tolist()[-111:] #Ceux sont les 111 dernières variables de notre base de données

In [133]:
dataSelect = data[VariblesPertinentes]

In [134]:
dataSelect.shape

(1386245, 111)

Apres la selection de nos variables pertinentes, notre base de données comporte bien 1.386.245 observations et 111 variables.

Parmi les nutriments présent dans notre base de données, nous avons des nutriments très rarement ou pas du tout présents dans les aliments. Nous choisirons de ne retenir que les nutriments qui sont présents dans au moins 10% des aliments. Ce sont alors les nutriments les plus couramment rencontrés dans les aliments. 

Pour determiner ces nutriments, nous allons tout d'abord determiner le pourcentage de données manquantes dans chaque variable. Ensuite, nous ne retiendrons dans notre étude que les variables qui auront moins de 90% de données manquantes.

In [135]:
#Pourcentage de données manquantes par varaible d'étude
POMD = (1 - dataSelect.count()/len(dataSelect)) *100
POMD_DF = pd.DataFrame(POMD)
POMD_DF.reset_index(level=0, inplace=True)
POMD_DF.columns = ['Variables', 'PercentageMD']
POMD_DF.sort_values(by=['PercentageMD'], inplace=True)

In [136]:
POMD_DF.head(10)

,Variables,PercentageMD
2,energy_100g,19.094604
53,proteins_100g,19.558808
4,fat_100g,19.655111
40,carbohydrates_100g,19.695364
57,salt_100g,20.859155
58,sodium_100g,20.859733
41,sugars_100g,21.188318
5,saturated-fat_100g,23.093537
1,energy-kcal_100g,24.205462
102,nutrition-score-fr_100g,59.035308


In [137]:
POMD_DF.tail(10)

,Variables,PercentageMD
6,-butyric-acid_100g,99.999928
37,-nervonic-acid_100g,100.000000
34,-gondoic-acid_100g,100.000000
19,-melissic-acid_100g,100.000000
36,-erucic-acid_100g,100.000000
105,water-hardness_100g,100.000000
7,-caproic-acid_100g,100.000000
35,-mead-acid_100g,100.000000
16,-lignoceric-acid_100g,100.000000
33,-elaidic-acid_100g,100.000000


In [138]:
POMD_DF['PercentageMD'].describe()

count    111.000000
mean      91.609426
std       22.009281
min       19.094604
25%       99.085768
50%       99.971001
75%       99.998377
max      100.000000
Name: PercentageMD, dtype: float64

Plus de 75% des nutriments dans notre base de données sont des nutriments rares.

In [139]:
NutrimentsRare = POMD_DF[POMD_DF['PercentageMD']>90]['Variables'].tolist()

In [140]:
NutCons = dataSelect.columns[~dataSelect.columns.isin(NutrimentsRare)].tolist()

In [141]:
dataSelect = dataSelect[NutCons]
dataSelect.shape

(1386245, 17)

Notre base de données comporte maintenant 17 variables quantitatives. Cependant, après la sélection de nos variables d'intérêt, nous pouvons nous retrouver avec des doublons dans la base de données. Nous allons supprimer une fois de plus ces doublons en vu de la segmentation.

In [142]:
dataSelect = dataSelect[dataSelect.duplicated()]
dataSelect.shape

(526787, 17)

Après suppression des doublons dans notre base de données, nous obtenons 526.787 individus et 17 variables. La suite consiste à l'analyse exploratoire des variables en vu d'optimiser la segmentation. 

In [144]:
# Savegardons la base de données dans un fichier csv pour l'analyse exploratoire.

dataSelect.to_csv('D:/IML/project2/DATA/dataSelect.csv', sep='\t', encoding='utf-8')

# FIN DE LA PARTIE NETTOYAGE DE LA BASE DE DONNEES